In [ ]:
import matplotlib.pyplot as plt
from time import time
import mediapipe as mp
import cv2 as cv
import math

In [2]:
mpPose = mp.solutions.pose
mpDraw = mp.solutions.drawing_utils

In [3]:
pose = mpPose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.3,
    model_complexity=2
)

In [4]:
def detectPose(image, pose, display=True):
    OutputImage = image.copy()
    imageRGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    results = pose.process(imageRGB)
    imgHeight, imgWidth, _ = image.shape
    landmarks = []

    if results.pose_landmarks:
        mpDraw.draw_landmarks(
            image=OutputImage,
            landmark_list=results.pose_landmarks,
            connections=mpPose.POSE_CONNECTIONS
        )
        for landmark in results.pose_landmarks.landmark:
            landmarks.append(
                (int(landmark.x * imgWidth),
                 int(landmark.y * imgHeight),
                 int(landmark.z * imgWidth))
            )
    if display:
        plt.figure(figsize=[22, 22])

        plt.subplot(211)
        plt.title('Original')
        plt.axis('off')
        plt.imshow(image[:, :, ::-1])

        plt.subplot(212)
        plt.title('Output')
        plt.axis('off')
        plt.imshow(OutputImage[:, :, ::-1])

        mpDraw.plot_landmarks(results.pose_world_landmarks, mpPose.POSE_CONNECTIONS)

    else:
        return OutputImage, landmarks

In [5]:
def calculateAngle(landmark1, landmark2, landmark3):

    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    if angle < 0:

        angle += 360
    
    return angle

In [ ]:
poseVideo = mpPose.Pose(
    static_image_mode=False,
    min_detection_confidence=0.5,
    model_complexity=1
)

video = cv.VideoCapture(0)

cv.namedWindow('Pose', cv.WINDOW_NORMAL)
video.set(3, 1280)
video.set(4, 960)
time1 = 0

while video.isOpened():
    ok, frame = video.read()
    
    if not ok:
        break

    frameHeight, frameWidth, _ = frame.shape
    frame = cv.resize(frame, (int(frameWidth * (640 / frameHeight)), 640))
    frame, ol = detectPose(frame, poseVideo, display=False)
    time2 = time()
    T = time2 - time1

    if T > 0:
        framesPerSecond = 1.0 / T
        if (calculateAngle(ol[24],ol[12],ol[14])<35 and calculateAngle(ol[24],ol[12],ol[14])>10 and calculateAngle(ol[12],ol[14],ol[16])>280 and calculateAngle(ol[12],ol[14],ol[16])<340) :   
            cv.putText(frame,'Hii', (950,40),cv.FONT_HERSHEY_PLAIN, 2, (0,255, 0), 3)
            
        if (calculateAngle(ol[24],ol[12],ol[14])<100 and calculateAngle(ol[24],ol[12],ol[14])>85 and calculateAngle(ol[12],ol[14],ol[16])>280 and calculateAngle(ol[12],ol[14],ol[16])<340
            and calculateAngle(ol[23],ol[11],ol[13])<340 and calculateAngle(ol[23],ol[11],ol[13])>265 and calculateAngle(ol[11],ol[13],ol[15])>20 and calculateAngle(ol[11],ol[13],ol[15])<60):   
            cv.putText(frame,'Dublebicep', (920,40),cv.FONT_HERSHEY_PLAIN, 2, (0,255, 0), 3)
            
        if (calculateAngle(ol[24],ol[12],ol[14])<18 and calculateAngle(ol[24],ol[12],ol[14])>10 and calculateAngle(ol[12],ol[14],ol[16])>168 and calculateAngle(ol[12],ol[14],ol[16])<190
            and calculateAngle(ol[23],ol[11],ol[13])<360 and calculateAngle(ol[23],ol[11],ol[13])>330 and calculateAngle(ol[11],ol[13],ol[15])>168 and calculateAngle(ol[11],ol[13],ol[15])<190
            and calculateAngle(ol[24],ol[26],ol[28])<183 and calculateAngle(ol[24],ol[26],ol[28])>176 and calculateAngle(ol[23],ol[25],ol[27])<183 and calculateAngle(ol[23],ol[25],ol[27])>178):
            cv.putText(frame,'STILL', (950,40),cv.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)

    time1 = time2
    cv.imshow('Pose New', frame)
    k = cv.waitKey(1) & 0xFF
    if k == 27:
        break

video.release()